In [37]:
import dgl
import torch
import torch.nn as nn
from dgl.nn import GINEConv,GATConv
import numpy as np
import time
from tqdm import tqdm

In [25]:
g=dgl.graph(([0,1,2,1],
             [1,2,1,0]))
in_feat=2
out_feat=2
graphs=[]
n_feat=torch.tensor([[1],[2],[3]])
e_feat=torch.randn(g.num_edges(),in_feat)
print(n_feat.shape)
print(e_feat.shape)

torch.Size([3, 1])
torch.Size([4, 2])


In [30]:
def get_nearest_neighbors(image, row, col):
    # 画像の形状を取得
    height, width = image.shape[:2]

    # 注目画素の周囲8画素の座標を計算
    neighbors_coords = [(row-1, col-1), (row-1, col), (row-1, col+1),
                        (row, col-1), (row, col+1),
                        (row+1, col-1), (row+1, col), (row+1, col+1)]

    # 注目画素の最近傍画素の値を抜き出す
    nearest_neighbors = []
    for r, c in neighbors_coords:
        # 座標が画像範囲内かチェック
        if 0 <= r < height and 0 <= c < width:
            pixel_value = image[r, c]
            nearest_neighbors.append(pixel_value)
        else:
            # 画像範囲外の場合は0を追加するなど適切な処理を行う
            #nearest_neighbors.append(0)
            pass

    return nearest_neighbors

In [33]:
def make_graph(side_length):
    g=dgl.DGLGraph()
    g.add_nodes(side_length**2)
    square_list = np.arange(side_length**2).reshape((side_length, side_length))
    #ノード番号に対応したインデックスを取得
    inds=np.ndindex(square_list.shape)
    inds=[idx for idx in inds]
    #各ノードと最近傍ノード間にエッジを張る
    for i in range(side_length**2):
        x,y=inds[i]
        flatt_nh=get_nearest_neighbors(square_list,x,y)
        for j in flatt_nh:
            if i == j:
                continue
            else:
                g.add_edges(j,i)
    return g

In [38]:
st=time.time()
for i in tqdm(range(5000)):
    g=make_graph(3)
print(time.time()-st)

  0%|          | 0/5000 [00:00<?, ?it/s]/home/echigo/anaconda3/envs/DGL3/lib/python3.9/site-packages/dgl/heterograph.py:92: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  dgl_warning(
100%|██████████| 5000/5000 [00:39<00:00, 127.66it/s]

39.20695900917053


In [32]:
print(n_feat)
print(e_feat)
conv=GINEConv(nn.Linear(in_feat,out_feat))
res=conv(g,n_feat,e_feat)
print(res)

tensor([[ 0.5888, -0.8675,  0.4436],
        [ 2.4353, -2.4259, -0.3092],
        [ 0.2220, -2.3916,  1.8348]])
tensor([[-0.4855,  0.0899, -1.2415],
        [ 1.0146, -0.0956,  0.1709],
        [ 0.9142,  0.9067, -0.5644],
        [-0.8427,  1.1041,  1.7460]])
tensor([[ 1.0899, -1.3193],
        [ 0.1359, -1.3522],
        [ 0.6151, -1.8083]], grad_fn=<AddmmBackward0>)


In [79]:
a=torch.randn((10,10,3))
b=torch.randn((10,10,3))
c=a.detach()

In [81]:
cos=nn.CosineSimilarity(0)
output=torch.cosine_similarity(a.flatten(),b.flatten(),dim=0)
print(output.item())

-0.09277930855751038


In [ ]:
g=dgl.graph(([0,1,2,1],
             [1,2,1,0]))
in_feat=2
out_feat=2
n_feat=torch.randn(g.num_nodes(),in_feat)
e_feat=torch.randn(g.num_edges(),in_feat)

In [67]:
gat1=GATConv(in_feat,10,2)
pred=gat1(g,n_feat,e_feat,get_attention=True)